In [183]:
import os

In [184]:
dataset_dir = os.path.join("", "dataset")
os.makedirs(dataset_dir, exist_ok=True)
print(os.listdir(dataset_dir))

updated_dataset_dir = os.path.join("", "updatedDataset")
os.makedirs(updated_dataset_dir, exist_ok=True)
print(os.listdir(updated_dataset_dir))
#4k format of height and width
H=440.
B=440.

['Test_label7.txt', 'Test_label10.txt', 'Test_label11.txt', 'Screenshot_Room7_LeftWall_Bathtub1_Faucet_Bathtub1_Floor1_Wall1_Time0_Camera_10.txt', 'Screenshot_Room7_LeftWall_Bathtub1_Faucet_Bathtub1_Floor1_Wall1_Time0_Camera_9.txt', 'Screenshot_Room7_LeftWall_Bathtub1_Faucet_Bathtub1_Floor1_Wall1_Time0_Camera_5 (1).txt', 'Test_label2.txt', 'Test_label14.txt', 'Screenshot_Room7_LeftWall_Bathtub1_Faucet_Bathtub1_Floor1_Wall1_Time0_Camera_7.txt', 'Test_label8.txt', '.ipynb_checkpoints', 'Screenshot_Room7_LeftWall_Bathtub1_Faucet_Bathtub1_Floor1_Wall1_Time0_Camera_0.txt', 'Test_label5.txt', 'Test_label12.txt', 'Test_label1.txt', 'Test_label3.txt', 'Test_label13.txt', 'Test_label6.txt', 'Screenshot_Room7_LeftWall_Bathtub1_Faucet_Bathtub1_Floor1_Wall1_Time0_Camera_5.txt', 'Test_label4.txt', 'Test_label9.txt']
['Test_label7.txt', 'Test_label10.txt', 'Test_label11.txt', 'Screenshot_Room7_LeftWall_Bathtub1_Faucet_Bathtub1_Floor1_Wall1_Time0_Camera_10.txt', 'Screenshot_Room7_LeftWall_Bathtub1_Fa

In [185]:
def get_label_names(folder_path, format=".txt"):
    # Check if the format is ".txt"
    if format == ".txt":
        # List comprehension to find all files ending with '.txt' in the specified folder
        txt_labels = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.txt')]
        # Return the list of txt file names
        return txt_labels


In [186]:
def get_label_content(path):
    # Initialize an empty list to hold the file content
    content = []

    # Open the file at the given path in read mode
    with open(path, "r") as fileref:
        # Iterate over each line in the file
        for i in fileref:
            # Append each line to the content list
            content.append(i)

    # Return the list containing all lines from the file
    return content

In [187]:
def change_label_content(path, updated_path):
    # Initialize variables to hold label index, coordinates, and sizes
    index = 0
    updated_x = 0
    updated_y = 0
    side_size_x = 0
    side_size_y = 0

    # Open the file at the given path in read mode
    with open(path, "r") as fileref:
        # Iterate over each line in the file
        for i in fileref:
            # Split the line into parts and parse the values as floats
            index = float(i.split(" ")[0])
            updated_x = float(i.split(" ")[1])
            updated_y = float(i.split(" ")[2])
            side_size_x = float(i.split(" ")[3])
            side_size_y = float(i.split(" ")[4])

    # Normalize the x and y coordinates
    updated_x = update_normalize_xy(updated_x, side_size_x, "X")
    updated_y = update_normalize_xy(updated_y, side_size_y, "Y")

    # Normalize the side sizes (assuming H and B are global or defined elsewhere)
    side_size_x = side_size_x / H
    side_size_y = side_size_y / B

    updated_x, updated_y, side_size_x, side_size_y = check_update_outliers(updated_x, updated_y, side_size_x, side_size_y)

    # Create a string with updated values concatenated
    st = f"{index} {updated_x} {updated_y} {side_size_x} {side_size_y}"

    # Open the file at the given path in write mode and write the updated string
    with open(updated_path, "w") as file:
        file.write(st)

In [188]:
def check_update_outliers(updated_x, updated_y, side_size_x, side_size_y):
    # Check if the right edge of the bounding box exceeds the right boundary (1.0)
    if updated_x + side_size_x / 2 > 1:
        # Adjust the x coordinate to bring the bounding box within the boundary
        updated_x = updated_x - (updated_x + side_size_x / 2 - 1) / 2
        # Adjust the width of the bounding box to fit within the boundary
        side_size_x = 2 * (1 - updated_x)
    # Check if the left edge of the bounding box exceeds the left boundary (0.0)
    elif updated_x - side_size_x / 2 < 0:
        # Adjust the x coordinate to bring the bounding box within the boundary
        updated_x = updated_x + (side_size_x / 2 - updated_x) / 2
        # Adjust the width of the bounding box to fit within the boundary
        side_size_x = 2 * updated_x

    # Check if the top edge of the bounding box exceeds the top boundary (1.0)
    if updated_y + side_size_y / 2 > 1:
        # Adjust the y coordinate to bring the bounding box within the boundary
        updated_y = updated_y - (updated_y + side_size_y / 2 - 1) / 2
        # Adjust the height of the bounding box to fit within the boundary
        side_size_y = 2 * (1 - updated_y)
    # Check if the bottom edge of the bounding box exceeds the bottom boundary (0.0)
    elif updated_y - side_size_y / 2 < 0:
        # Adjust the y coordinate to bring the bounding box within the boundary
        updated_y = updated_y + (side_size_y / 2 - updated_y) / 2
        # Adjust the height of the bounding box to fit within the boundary
        side_size_y = 2 * updated_y

    # Check for the corners where both X and Y boundaries are exceeded
    # Bottom-left corner
    if (updated_x - side_size_x / 2 < 0) and (updated_y - side_size_y / 2 < 0):
        # Adjust both x and y coordinates to bring the bounding box within the boundary
        updated_x = side_size_x / 2
        updated_y = side_size_y / 2

    # Bottom-right corner
    if (updated_x + side_size_x / 2 > 1) and (updated_y - side_size_y / 2 < 0):
        # Adjust both x and y coordinates to bring the bounding box within the boundary
        updated_x = 1 - side_size_x / 2
        updated_y = side_size_y / 2

    # Top-left corner
    if (updated_x - side_size_x / 2 < 0) and (updated_y + side_size_y / 2 > 1):
        # Adjust both x and y coordinates to bring the bounding box within the boundary
        updated_x = side_size_x / 2
        updated_y = 1 - side_size_y / 2

    # Top-right corner
    if (updated_x + side_size_x / 2 > 1) and (updated_y + side_size_y / 2 > 1):
        # Adjust both x and y coordinates to bring the bounding box within the boundary
        updated_x = 1 - side_size_x / 2
        updated_y = 1 - side_size_y / 2

    # Return the potentially adjusted coordinates and sizes
    return updated_x, updated_y, side_size_x, side_size_y

In [189]:
#One with 4 checks
"""def check_update_outliers(updated_x, updated_y, side_size_x, side_size_y):
    # Check if the square exceeds the right boundary
    if updated_x + side_size_x / 2 > 1:
        updated_x = updated_x - (updated_x + side_size_x / 2 - 1) / 2
        side_size_x = 2 * (1 - updated_x)
    # Check if the square exceeds the left boundary
    elif updated_x - side_size_x / 2 < 0:
        updated_x = updated_x + (side_size_x / 2 - updated_x) / 2
        side_size_x = 2 * updated_x

    # Check if the square exceeds the top boundary
    if updated_y + side_size_y / 2 > 1:
        updated_y = updated_y - (updated_y + side_size_y / 2 - 1) / 2
        side_size_y = 2 * (1 - updated_y)
    # Check if the square exceeds the bottom boundary
    elif updated_y - side_size_y / 2 < 0:
        updated_y = updated_y + (side_size_y / 2 - updated_y) / 2
        side_size_y = 2 * updated_y

    return updated_x, updated_y, side_size_x, side_size_y"""

'def check_update_outliers(updated_x, updated_y, side_size_x, side_size_y):\n    # Check if the square exceeds the right boundary\n    if updated_x + side_size_x / 2 > 1:\n        updated_x = updated_x - (updated_x + side_size_x / 2 - 1) / 2\n        side_size_x = 2 * (1 - updated_x)\n    # Check if the square exceeds the left boundary\n    elif updated_x - side_size_x / 2 < 0:\n        updated_x = updated_x + (side_size_x / 2 - updated_x) / 2\n        side_size_x = 2 * updated_x\n    \n    # Check if the square exceeds the top boundary\n    if updated_y + side_size_y / 2 > 1:\n        updated_y = updated_y - (updated_y + side_size_y / 2 - 1) / 2\n        side_size_y = 2 * (1 - updated_y)\n    # Check if the square exceeds the bottom boundary\n    elif updated_y - side_size_y / 2 < 0:\n        updated_y = updated_y + (side_size_y / 2 - updated_y) / 2\n        side_size_y = 2 * updated_y\n\n    return updated_x, updated_y, side_size_x, side_size_y'

In [190]:
def update_normalize_xy(param, side_size, dimension="X", H=440., B=440.):
    """
    Normalizes the param value for the specified dimension (X or Y).

    Args:
    param (float): The parameter value to normalize.
    side_size (float): The size to be added for centering.
    dimension (str): The dimension to normalize ('X' or 'Y').
    H (float): The maximum range for X dimension (default is 3840.0).
    B (float): The maximum range for Y dimension (default is 2160.0).

    Returns:
    float: The normalized parameter value in the range [0, 1].

    Raises:
    ValueError: If the dimension is not 'X' or 'Y'.
    """
    if dimension == "X":
        # Normalize the param value for the X dimension
        return (param + side_size / 2) / H
    elif dimension == "Y":
        # Normalize the param value for the Y dimension
        return (param + side_size / 2) / B
    else:
        raise ValueError("Dimension must be 'X' or 'Y'")

In [191]:
get_label_names(dataset_dir, format=".txt")
for i in get_label_names(dataset_dir, format=".txt"):
  path = dataset_dir + "/" + i
  path = "dataset" + "/" + i
  updated_path = "updatedDataset" + "/" + i
  change_label_content(path, updated_path)